In [5]:
import pandas as pd

df_train = pd.read_csv('./train_full.csv')
df_validation = pd.read_csv('./validation_full.csv')

In [6]:
X_train, X_val, y_train, y_val = df_train.iloc[:,0:-1], df_validation.iloc[:,0:-1], df_train['label'].values*1, df_validation['label'].values*1

In [7]:
y_train = [10 if x==11 else x for x in y_train]
y_val = [10 if x==11 else x for x in y_val]

In [8]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_train_enc = np_utils.to_categorical(y_train_enc)
y_val_enc = encoder.fit_transform(y_val)
y_val_enc = np_utils.to_categorical(y_val)

In [18]:
from keras.layers import Dropout

def baseline_model():
  model = Sequential()
  model.add(Dense(2048, input_dim=2925, kernel_initializer='normal', activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(1024, kernel_initializer='normal', activation='relu'))
  model.add(Dense(512, kernel_initializer='normal', activation='relu'))
  model.add(Dense(128, kernel_initializer='normal', activation='relu'))
  model.add(Dense(11, kernel_initializer='normal', activation='softmax'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [19]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=17, batch_size=100, verbose=1 ,)
estimator.fit(X_train, y_train_enc)

Epoch 1/17
51157/51157 [==============================] - 12s 231us/step - loss: 1.1381 - acc: 0.6798
Epoch 2/17
51157/51157 [==============================] - 11s 222us/step - loss: 0.6845 - acc: 0.7873
Epoch 3/17
51157/51157 [==============================] - 11s 220us/step - loss: 0.5368 - acc: 0.8316
Epoch 4/17
51157/51157 [==============================] - 11s 224us/step - loss: 0.4448 - acc: 0.8605
Epoch 5/17
51157/51157 [==============================] - 11s 223us/step - loss: 0.3955 - acc: 0.8760
Epoch 6/17
51157/51157 [==============================] - 11s 221us/step - loss: 0.3493 - acc: 0.8904
Epoch 7/17
51157/51157 [==============================] - 11s 224us/step - loss: 0.3164 - acc: 0.9010
Epoch 8/17
51157/51157 [==============================] - 11s 223us/step - loss: 0.2878 - acc: 0.9110
Epoch 9/17
51157/51157 [==============================] - 11s 225us/step - loss: 0.2628 - acc: 0.9169
Epoch 10/17
51157/51157 [==============================] - 12s 227us/step - loss: 

In [15]:
train_pred = estimator.predict(X_train)
val_pred = estimator.predict(X_val)
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
print('Train Class Accuracy:' + str(accuracy_score(y_train, train_pred)))
print('Test Class Accuracy:' + str(accuracy_score(y_val, val_pred)))
pd.DataFrame(confusion_matrix(y_val, val_pred))

5895/5895 [==============================] - 0s 71us/step
Train Class Accuracy:0.943761362081
Test Class Accuracy:0.870568278202


,0,1,2,3,4,5,6,7,8,9,10
0,190,1,0,0,4,0,0,0,0,0,27
1,3,156,3,5,0,1,0,0,0,8,50
2,0,1,166,0,0,0,0,2,2,0,39
3,0,11,2,154,1,0,8,0,0,4,58
4,10,1,2,0,170,0,0,3,0,0,37
5,1,0,1,0,15,127,0,1,0,0,74
6,0,0,4,0,0,0,173,4,0,1,43
7,1,0,13,0,1,0,1,177,0,0,27
8,0,0,4,1,0,0,0,0,172,0,36
9,0,36,5,14,0,0,1,0,0,110,53


In [44]:
pd.DataFrame(confusion_matrix(y_train, train_pred))

,0,1,2,3,4,5,6,7,8,9,10
0,1789,0,0,0,17,0,0,1,0,0,87
1,1,1620,0,13,1,0,0,1,0,55,136
2,0,1,1525,0,0,0,7,72,17,5,177
3,0,20,0,1655,0,0,4,0,0,6,178
4,6,0,0,0,1753,3,0,19,0,0,120
5,0,0,0,0,14,1634,0,1,0,0,227
6,0,0,0,5,0,0,1670,7,0,0,158
7,0,0,5,0,0,1,19,1730,0,0,106
8,0,1,1,0,1,0,0,2,1784,0,127
9,0,119,3,29,0,0,2,0,1,1471,198
